In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
cp /content/drive/MyDrive/base_model/kvasir-dataset-v2.zip .

In [3]:
! 7z x kvasir-dataset-v2.zip -aoa -o/content/


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 2489312085 bytes (2374 MiB)

Extracting archive: kvasir-dataset-v2.zip
 51% 4096 Open              --
Path = kvasir-dataset-v2.zip
Type = zip
Physical Size = 2489312085

  0%      0% 46 - kvasir-dataset-v2/polyps/11ad2a58-e96d-4cd4-b3af-d18ead3547c7.jpg                                                                             0% 90 - kvasir-dataset-v2/polyps/9a1f8fc3-c7d2-4a11-b403-94d95ae2f4c6.jpg                                                             

In [4]:
import os
import numpy as np
import seaborn as sea
import tensorflow as tf 
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow.keras.applications as Applications
import tensorflow.keras.preprocessing.image as ImagePreprocessing
import cv2
import pickle
import json
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dense ,MaxPool2D ,BatchNormalization,Flatten ,Dropout
from sklearn.model_selection import KFold
from tensorflow.keras.applications.resnet50 import ResNet50
from PIL import Image
import importlib.util
import random as rn
import pandas as pd
from sklearn.model_selection import train_test_split
from shutil import copyfile
from matplotlib import pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
import json
import tensorflow.keras as keras

sns.set()

In [5]:
train_split = 0.7
val_split = 0.15
test_split = 0.15
original_ds_path = 'kvasir-dataset-v2'
train_ds_path = 'train'
val_ds_path = 'validate'
test_ds_path = 'test'
base_dir = '/content/drive/MyDrive/base_model/'
losses_dir = base_dir+'losses/'
model_save= base_dir+'model_saves/'
seed = 49
size_img = (224,224) #size choosen to preserve the aspect ratio
input_size = (224,224,3)
batch_size = 16

In [6]:
ds_dic = {}
for folder in os.listdir(original_ds_path):
  ds_dic[folder] = os.listdir(original_ds_path+'/'+folder) 

In [7]:
if not os.path.exists(train_ds_path):
      os.mkdir(train_ds_path)
if not os.path.exists(val_ds_path):
      os.mkdir(val_ds_path)
if not os.path.exists(test_ds_path):
      os.mkdir(test_ds_path)

for each in ds_dic:
    
    if not os.path.exists(train_ds_path+'/'+each+'/'):
        os.mkdir(train_ds_path+'/'+each+'/')
    if not os.path.exists(val_ds_path+'/'+each+'/'):
        os.mkdir(val_ds_path+'/'+each+'/')
    if not os.path.exists(test_ds_path+'/'+each+'/'):
        os.mkdir(test_ds_path+'/'+each+'/')

In [8]:
for each in ds_dic.keys():
  train_split , intermediate_split=train_test_split(ds_dic[each], 
                                              test_size=0.3, 
                                              random_state=seed,
                                              shuffle = True)
  
  val_split , test_split = train_test_split(intermediate_split, 
                                              test_size=0.5, 
                                              random_state=seed,
                                              shuffle = True)
  

  #print(len(train_split))
  #print(len(test_split))
  #print(len(val_split))

  for img in train_split:
        destination = train_ds_path+'/'+each+'/'+img
        source = original_ds_path+'/'+each+'/'+img
        
        if not os.path.exists(destination):
            copyfile(source, destination)
        
  for img in test_split:
      destination = test_ds_path+'/'+each+'/'+img
      source = original_ds_path+'/'+each+'/'+img
      
      if not os.path.exists(destination):
          copyfile(source, destination)

  for img in val_split:
      destination = val_ds_path+'/'+each+'/'+img
      source = original_ds_path+'/'+each+'/'+img
      
      if not os.path.exists(destination):
          copyfile(source, destination)

In [9]:
train_datagen = ImageDataGenerator(featurewise_center=True,
                               samplewise_center=True,
                               featurewise_std_normalization=True,
                               samplewise_std_normalization=True,
                               zca_whitening=False,
                               zca_epsilon=1e-06,
                               rotation_range=180,
                               width_shift_range=0.0,
                               height_shift_range=0.0, 
                               brightness_range=None, 
                               shear_range=0.0, 
                               zoom_range=[0.0,2.0],
                               channel_shift_range=0.0, 
                               fill_mode='nearest', 
                               cval=0.0, 
                               horizontal_flip=True,
                               vertical_flip=True, 
                               rescale=1./255, 
                               preprocessing_function=None,
                               validation_split=0.15
    )

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [10]:
train_generator = train_datagen.flow_from_directory(directory=train_ds_path,
                                                    target_size=size_img,
                                                    batch_size=batch_size,
                                                    shuffle=True, 
                                                    seed=seed,
                                                    )
val_generator = val_datagen.flow_from_directory(directory=val_ds_path,
                                                    target_size=size_img,
                                                    batch_size=batch_size,
                                                    shuffle=True, 
                                                    seed=seed,
                                                    )
test_generator = test_datagen.flow_from_directory(directory=test_ds_path,
                                                    target_size=size_img,
                                                    batch_size=batch_size,
                                                    shuffle=True, 
                                                    seed=seed,
                                                    )

Found 5600 images belonging to 8 classes.
Found 1200 images belonging to 8 classes.
Found 1200 images belonging to 8 classes.


In [11]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
#tf.debugging.set_log_device_placement(False)

Num GPUs Available:  1


In [12]:
model = keras.Sequential()
model.add(Conv2D(512,(3,3),activation='relu',input_shape = input_size))
model.add(MaxPool2D(2,2))
model.add(Conv2D(512,(3,3),activation='relu'))
model.add(Dropout(0.2))
model.add(MaxPool2D(2,2))
model.add(BatchNormalization())
model.add(Conv2D(512,(3,3),activation='relu'))
model.add(Dropout(0.2))
model.add(MaxPool2D(2,2))
model.add(Conv2D(512,(3,3),activation='relu'))
model.add(MaxPool2D(2,2))
model.add(BatchNormalization())
#model.add(Conv2D(1024,(3,3),activation='relu'))
#model.add(MaxPool2D(2,2))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(8,activation='softmax'))

In [13]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 512)     14336     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 512)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 512)     2359808   
_________________________________________________________________
dropout (Dropout)            (None, 109, 109, 512)     0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 512)       0         
_________________________________________________________________
batch_normalization (BatchNo (None, 54, 54, 512)       2048      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 512)       2

In [14]:
model.compile(optimizer=tf.keras.optimizers.Adamax(lr=1e-2), 
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
                       tf.keras.metrics.CategoricalCrossentropy(name='crossentropy_loss'),
                       ]
              )

In [15]:
%ls /content/drive/MyDrive/base_model/

Basic_CNN.ipynb  kvasir-dataset-v2.zip  model_saves/


In [16]:
import os
try:
  if (not os.path.exists(base_dir+'model_saves/')):
    os.mkdir(base_dir+'model_saves/')
except FileExistsError:
  print('File Exists')
  

In [17]:
#model.save_weights(base_dir+'model_saves/'+str('1')+'_init.h5')

In [18]:
train_history_list = {'initial':[],'secondary':[]}
val_history_list = {'initial':[],'secondary':[]}


In [19]:

history = model.fit_generator(
    generator=train_generator, steps_per_epoch=None, epochs=50, verbose=1, callbacks=None,
    validation_data=val_generator,
    shuffle=True, initial_epoch=0)

train_history_list['initial'].append(history)

model.save_weights(model_save+str(times)+'_init.h5')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/50
350/350 [==============================] - 262s 618ms/step - loss: 26.8966 - accuracy: 0.2331 - crossentropy_loss: 26.8966 - val_loss: 2.7172 - val_accuracy: 0.1867 - val_crossentropy_loss: 2.7172
Epoch 2/50
350/350 [==============================] - 215s 613ms/step - loss: 1.7830 - accuracy: 0.2925 - crossentropy_loss: 1.7830 - val_loss: 4.2155 - val_accuracy: 0.1500 - val_crossentropy_loss: 4.2155
Epoch 3/50
350/350 [==============================] - 214s 612ms/step - loss: 1.6355 - accuracy: 0.3307 - crossentropy_loss: 1.6355 - val_loss: 2.4253 - val_accuracy: 0.2317 - val_crossentropy_loss: 2.4253
Epoch 4/50
350/350 [==============================] - 216s 616ms/step - loss: 1.5799 - accuracy: 0.3441 - crossentropy_loss: 1.5799 - val_loss: 11.0643 - val_accuracy: 0.1325 - val_crossentropy_loss: 11.0643
Epoch 5/50
350/350 [==============================] - 215s 613ms/step - loss: 1.5101 - accuracy: 0.3733 - crossentropy_loss: 1.5101 - val_loss: 2.0120 - val_accuracy: 0.232

NameError: ignored